In [3]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 로더 설정
loaders = [TextLoader("data/How_to_invest_money.txt", encoding="utf-8")]

docs = []
for loader in loaders:
    docs.extend(loader.load())

In [5]:
# 문서 생성을 위한 텍스트 분할기 정의
recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 문서 분할
split_docs = recursive_splitter.split_documents(docs)

# Ollama 임베딩 모델 설정
embedding = OllamaEmbeddings(model="bge-m3")

# Chroma vectorstore 생성
vectorstore = Chroma.from_documents(documents=split_docs, embedding=embedding)

# Chroma vectorstore 기반 리트리버 생성
retriever = vectorstore.as_retriever()

- HyDE 방식을 구현하는 코드를 살펴보겠습니다. 
- 랭체인의 체인을 사용합니다. 각 체인은 자신의 역할에 집중하면서도 전체 파이프라인에서는 유기적으로 결합되어 효울적으로 작업을 처리할 수 있습니다.
- 가상 문서 생성 체인
- 문서 검색 체인
- 최종 응답 생성 체인

#### 가상 문서 생성 체인
1. 프롬프트: 시스템 메시지와 사용자 메시지를 정의한 뒤 이들을 ChatPromptTemplate에 넣어 LLM에 전달할 프롬프트를 생성합니다.
2. LLM : ChatOpenAI를 통해 입력된 프롬프트에 기반해 가상의 문서를 생성합니다. 
3. 파서 : LLM의 출력을 문자열 형태로 변한하기 위해 StrOutputParser를 사용합니다.

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_text_splitters import CharacterTextSplitter

# 1. 가상 문서 생성 체인
def create_virtual_doc_chain():
    system = "당신은 고도로 숙련된 AI입니다."
    user = """
    주어진 질문 '{query}' 에 대해 직접적으로 답변하는 가상의 문서를 생성하세요.
    문서의 크기는 {chunk_size} 글자 언저리여야 합니다.
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        ("user", user)
    ])
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
    return prompt | llm | StrOutputParser()

In [13]:
# 2. 문서 검색 체인
def create_retrieval_chain():
    return RunnableLambda(lambda x: retriever.invoke(x['virtual_doc']))

# 유틸리티 함수
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [8]:
# 3. 최종 응답 생성 체인
def create_final_response_chain():
    final_prompt = ChatPromptTemplate.from_template("""
    다음 정보와 질문을 바탕으로 답변해주세요:
    컨텍스트: {context}
    질문: {question}
    답변:
    """)
    final_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
    return final_prompt | final_llm

### RunnableLambda 메서드
- 사용자 정의 함수를 랭체인의 실행 가능한 객체로 변환하는 데 사용됩니다.
- 복잡한 로직을 포함하는 사용자 정의 함수를 파이프라인의 다른 구성요소와 쉽게 통합할 수 있습니다.

In [9]:
from langchain_core.runnables import RunnableLambda

# 나이를 받아 성인 여부를 판단하는 함수
def check_adult(input_dict):
    return {"adult_status": "성인" if input_dict["age"] >= 20 else "미성년자"}

# RunnableLambda를 사용한 체인 구성
chain = RunnableLambda(check_adult)

# 체인 실행
result = chain.invoke({"name": "홍길동", "age": 25})
print(result)  # {'adult_status': '성인'}

{'adult_status': '성인'}


- check_adult 함수는 RunnableLambda를 통해 랭체인의 실행 가능한 객체로 변환됩니다.
- 이 함수는 딕셔너리를 받아 나이를 확인하고 성인 여부를 나타내는 새로운 딕셔너리를 반환합니다.

In [10]:
from langchain_core.runnables import RunnableLambda
def print_input_output(input_data, output_data, step_name):
    print(f"\n--- {step_name} ---")
    print(f"Input: {input_data}")
    print(f"Output: {output_data}")
    print("-" * 50)

- HyDE 방식을 구현하는 전체 파이프라인을 생성하는 메인 함수를 만들어 보겠습니다.

In [11]:
def create_pipeline_with_logging():
    # 가상 문서 생성 체인
    virtual_doc_chain = create_virtual_doc_chain()
    # 문서 검색 체인
    retrieval_chain = create_retrieval_chain()
    # 최종 응답 생성 체인
    final_response_chain = create_final_response_chain()

    # 가상 문서 생성 단계
    def virtual_doc_step(x):
        result = {"virtual_doc": virtual_doc_chain.invoke({
            "query": x['question'],
            "chunk_size": 200
        })}
        print_input_output(x, result, "Virtual Document Generation")
        return {**x, **result}
    
    # 문서 검색 단계
    def retrieval_step(x):
        result = {"retrieved_docs": retrieval_chain.invoke(x)}
        print_input_output(x, result, "Document Retrieval")
        return {**x, **result}
    
    # 컨텍스트 포매팅 단계
    def context_formatting_step(x):
        result = {"context": format_docs(x['retrieved_docs'])}
        print_input_output(x, result, "Context Formatting")
        return {**x, **result}
    
    # 최종 응답 생성 단계
    def final_response_step(x):
        result = final_response_chain.invoke(x)
        print_input_output(x, result, "Final Response Generation")
        return result
    
    # 전체 파이프라인 구성
    pipeline = (
        RunnableLambda(virtual_doc_step)
        | RunnableLambda(retrieval_step)
        | RunnableLambda(context_formatting_step)
        | RunnableLambda(final_response_step)
    )

    return pipeline

# 파이프라인 객체 생성
pipeline = create_pipeline_with_logging()
            

## 1. 가상 문서 생성 단계: virtual_doc_step 함수
- 사용자의 질문을 바탕으로 가상의 문서를 생성합니다.
- 입력값은 사용자의 질문을 포함하는 딕셔너리입니다.
```json
{"question": "주식 시장의 변동성이 높을 때 투자 전략은 무엇인가요?"}
```
- virtual_doc_chain.invoke() 메서드를 호출하여 가상 문서를 생성합니다.
- 가상 문서가 생성되면 기존 입력값과 병합하여 반환합니다. 이는 다음 단계인 문서 검색 과정으로 넘어가게 됩니다.

## 2. 문서 검색 단계: retrieval_step함수
- 가상 문서를 기반으로 벡터 데이터베이스에서 관련 문서를 검색합니다.
- 입력값은 가상문서와 원본 질문이 포함된 딕셔너리 x입니다.
- retrieval_chain.invoke(x)를 호출하여 수행됩니다.
- retrieval.get_relevant_documents로 가상 문서를 활용한 관련 문서 검색이 이루어집니다.
- 검색이 완료되면 생성되는 retrieved_docs를 기존 입력값 x와 병합하여 다음단계로 전달합니다.

## 3. 컨텍스트 포매팅 단계: context_formatting_step함수
- 검색된 문서들을 하나의 문자열로 포매팅하여 최종 답변 생성에 사용할 컨텍스트를 만듭니다.
- 입력값은 앞 단계에서 검색된 문서들이 포함된 딕셔너리 x입니다.
- format_docs(x["retrieved_docs"]) 함수를 호출하여 진행됩니다.
- 생성된 컨텍스트는 기존 입력값과 병합되어 반환되며, 최종 응답 생성을 위한 단계로 전달됩니다.

## 4. 최종 응답 생성 단계: final_response_step함수
- 앞서 생성된 컨텍스트와 원본 질문을 바탕으로 최종 답변을 생성합니다.
- 입력값은 컨텍스트와 원본 질문을 포함한 딕셔너리 x입니다.
- final_response_chain.invoke(x)가 호출되며, 이때 LLM이 동작하여 입력된 컨텍스트와 질문을 기반으로 최종 답변을 생성하여 반환합니다.

## 5. 전체 파이프라인 구성
- 모든 단계를 순차적으로 연결하여 전체 파이프라인을 완성하는 단계입니다.
- virtual_doc_step -> retrieval_step -> context_formatting_step -> final_response_step 순으로 연결됩니다.

- 파이프라인 첫 단계인 virtual_doc_step 함수가 question 키를 기준으로 동작하므로 질문은 {"question": question} 형태의 딕셔너리여야 합니다.

In [14]:
# 질문과 답변 예
question = "주식 시장의 변동성이 높을 때 투자 전략은 무엇인가요?"
response = pipeline.invoke({"question": question})
print(f"최종 답변: {response.content}")


--- Virtual Document Generation ---
Input: {'question': '주식 시장의 변동성이 높을 때 투자 전략은 무엇인가요?'}
Output: {'virtual_doc': '**주식 시장의 변동성이 높을 때 투자 전략**\n\n변동성이 큰 주식 시장에서는 신중한 접근이 필요합니다. 첫째, 분산 투자로 리스크를 줄이는 것이 중요합니다. 다양한 자산군에 투자하여 특정 주식의 하락에 대한 영향을 최소화하세요. 둘째, 방어적인 주식이나 배당주에 집중하는 것도 좋은 전략입니다. 이러한 주식은 시장 불안정성에도 상대적으로 안정적인 수익을 제공합니다. 셋째, 손절매 주문을 설정하여 손실을 제한하는 것도 고려해보세요. 마지막으로, 시장의 흐름을 주의 깊게 관찰하고, 감정에 휘둘리지 않도록 냉정함을 유지하는 것이 중요합니다.'}
--------------------------------------------------

--- Document Retrieval ---
Input: {'question': '주식 시장의 변동성이 높을 때 투자 전략은 무엇인가요?', 'virtual_doc': '**주식 시장의 변동성이 높을 때 투자 전략**\n\n변동성이 큰 주식 시장에서는 신중한 접근이 필요합니다. 첫째, 분산 투자로 리스크를 줄이는 것이 중요합니다. 다양한 자산군에 투자하여 특정 주식의 하락에 대한 영향을 최소화하세요. 둘째, 방어적인 주식이나 배당주에 집중하는 것도 좋은 전략입니다. 이러한 주식은 시장 불안정성에도 상대적으로 안정적인 수익을 제공합니다. 셋째, 손절매 주문을 설정하여 손실을 제한하는 것도 고려해보세요. 마지막으로, 시장의 흐름을 주의 깊게 관찰하고, 감정에 휘둘리지 않도록 냉정함을 유지하는 것이 중요합니다.'}
Output: {'retrieved_docs': [Document(id='1e7e586f-4add-40e1-8698-2eb780fb8b94', metadata={'source': 'data/How_to_invest_m